In [1]:
import pandas as pd

lat_grid_size = 90
lon_grid_size = 180

records = []
for max_lat in range(90, -90, -lat_grid_size):
    min_lat = max_lat - lat_grid_size
    for min_lon in range(-180, 180, lon_grid_size):
        max_lon = min_lon + lon_grid_size
        records.append([max_lat, min_lat, max_lon, min_lon])

df_grid = pd.DataFrame(records, columns=["max_lat", "min_lat", "max_lon", "min_lon"])

records = []
for end_year in range(2020, 1940, -40):
    start_year = end_year - 39
    records.append([start_year, end_year])
df_year = pd.DataFrame(records, columns=["start_year", "end_year"])

variables = ["temperature", "precipitation", "pressure", "wind"]
df_var = pd.DataFrame(variables, columns=["variable"])

df = pd.merge(df_var, df_year, how="cross")
df_ui = pd.merge(df, df_grid, how="cross")
import random

temporal_resolutions = ["hour", "hour", "day", "day", "day", "day", "day", "month", "year"]
spatial_resolutions = [0.25, 0.25, 0.5, 0.5, 1]

df_ui["temporal_resolution"] = [random.choice(temporal_resolutions) for _ in range(len(df_ui))]
df_ui["spatial_resolution"] = [random.choice(spatial_resolutions) for _ in range(len(df_ui))]
df_ui["ui_id"] = df_ui.index + 1
df_ui

,variable,start_year,end_year,max_lat,min_lat,max_lon,min_lon,temporal_resolution,spatial_resolution,ui_id
0,temperature,1981,2020,90,0,0,-180,day,0.50,1
1,temperature,1981,2020,90,0,180,0,month,0.50,2
2,temperature,1981,2020,0,-90,0,-180,day,0.25,3
3,temperature,1981,2020,0,-90,180,0,day,0.25,4
4,temperature,1941,1980,90,0,0,-180,month,0.50,5
5,temperature,1941,1980,90,0,180,0,hour,0.25,6
6,temperature,1941,1980,0,-90,0,-180,day,0.25,7
7,temperature,1941,1980,0,-90,180,0,day,0.25,8
8,precipitation,1981,2020,90,0,0,-180,hour,0.50,9
9,precipitation,1981,2020,90,0,180,0,day,0.25,10


In [2]:
def calc_data_size(start_year, end_year, max_lat, min_lat, max_lon, min_lon, t_res, s_res):
    lat_size = (max_lat - min_lat) / s_res
    lon_size = (max_lon - min_lon) / s_res
    if t_res == "hour":
        time_size = (end_year - start_year) * 365 * 24
    elif t_res == "day":
        time_size = (end_year - start_year) * 365
    elif t_res == "month":
        time_size = (end_year - start_year) * 12
    elif t_res == "year":
        time_size = end_year - start_year

    byte_size = lat_size * lon_size * time_size * 1.7
    gb_size = byte_size / 1024 / 1024 / 1024
    if s_res != 0.25 or t_res != "hour":
        gb_size = gb_size * 3  # coarse resolution has min, max, and mean aggregation
    return gb_size


df_ui["size"] = df_ui.apply(
    lambda x: calc_data_size(
        x["start_year"],
        x["end_year"],
        x["max_lat"],
        x["min_lat"],
        x["max_lon"],
        x["min_lon"],
        x["temporal_resolution"],
        x["spatial_resolution"],
    ),
    axis=1,
)
print(df_ui["size"].sum(), "GB")
df_ui

1009.7151146829128 GB


,variable,start_year,end_year,max_lat,min_lat,max_lon,min_lon,temporal_resolution,spatial_resolution,ui_id,size
0,temperature,1981,2020,90,0,0,-180,day,0.50,1,4.381298
1,temperature,1981,2020,90,0,180,0,month,0.50,2,0.144043
2,temperature,1981,2020,0,-90,0,-180,day,0.25,3,17.525192
3,temperature,1981,2020,0,-90,180,0,day,0.25,4,17.525192
4,temperature,1941,1980,90,0,0,-180,month,0.50,5,0.144043
5,temperature,1941,1980,90,0,180,0,hour,0.25,6,140.201533
6,temperature,1941,1980,0,-90,0,-180,day,0.25,7,17.525192
7,temperature,1941,1980,0,-90,180,0,day,0.25,8,17.525192
8,precipitation,1981,2020,90,0,0,-180,hour,0.50,9,105.151150
9,precipitation,1981,2020,90,0,180,0,day,0.25,10,17.525192


In [3]:
def create_meta_record(row):
    new_meta_records = []
    all_temporal_resolutions = ["hour", "day", "month", "year"]
    all_spatial_resolutions = [0.25, 0.5, 1]
    t_res = row.temporal_resolution
    s_res = row.spatial_resolution
    for tr in all_temporal_resolutions[all_temporal_resolutions.index(t_res) :]:
        for sr in all_spatial_resolutions[all_spatial_resolutions.index(s_res) :]:
            new_meta_records.append(
                [
                    row.ui_id,
                    row.variable,
                    row.start_year,
                    row.end_year,
                    row.max_lat,
                    row.min_lat,
                    row.max_lon,
                    row.min_lon,
                    row.temporal_resolution,
                    row.spatial_resolution,
                    tr,
                    sr,
                ]
            )
    return new_meta_records

meta_records = []
for row in df_ui.itertuples():
    new_records = create_meta_record(row)
    meta_records += new_records

df_meta = pd.DataFrame(
    meta_records,
    columns=[
        "ui_id",
        "variable",
        "start_year",
        "end_year",
        "max_lat",
        "min_lat",
        "max_lon",
        "min_lon",
        "ui_temporal_resolution",
        "ui_spatial_resolution",
        "actual_temporal_resolution",
        "actual_spatial_resolution",
    ],
)

df_meta['size'] = df_meta.apply(
    lambda x: calc_data_size(
        x["start_year"],
        x["end_year"],
        x["max_lat"],
        x["min_lat"],
        x["max_lon"],
        x["min_lon"],
        x["actual_temporal_resolution"],
        x["actual_spatial_resolution"],
    ),
    axis=1,
)
print(df_meta["size"].sum(), "GB")
df_meta

1541.5536647848785 GB


,ui_id,variable,start_year,end_year,max_lat,min_lat,max_lon,min_lon,ui_temporal_resolution,ui_spatial_resolution,actual_temporal_resolution,actual_spatial_resolution,size
0,1,temperature,1981,2020,90,0,0,-180,day,0.5,day,0.5,4.381298
1,1,temperature,1981,2020,90,0,0,-180,day,0.5,day,1.0,1.095324
2,1,temperature,1981,2020,90,0,0,-180,day,0.5,month,0.5,0.144043
3,1,temperature,1981,2020,90,0,0,-180,day,0.5,month,1.0,0.036011
4,1,temperature,1981,2020,90,0,0,-180,day,0.5,year,0.5,0.012004
...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,32,wind,1941,1980,0,-90,180,0,day,0.5,day,1.0,1.095324
216,32,wind,1941,1980,0,-90,180,0,day,0.5,month,0.5,0.144043
217,32,wind,1941,1980,0,-90,180,0,day,0.5,month,1.0,0.036011
218,32,wind,1941,1980,0,-90,180,0,day,0.5,year,0.5,0.012004


In [4]:
import duckdb

sql = """
SELECT u.*, g.agg_size, g.agg_size / u.size as ratio
FROM df_ui u, (
    SELECT u.ui_id, sum(m.size) as agg_size
    FROM df_ui u, df_meta m
    WHERE u.ui_id = m.ui_id
    GROUP BY u.ui_id
) g
WHERE u.ui_id = g.ui_id
"""
df_ui_with_size = duckdb.query(sql).df()
ui_size = df_ui_with_size["size"].sum()
actual_size = df_ui_with_size["agg_size"].sum()
print("ui only size:", ui_size, "GB")
print("including pre-aggregation size:", actual_size, "GB")
print("ratio:", actual_size / ui_size)
df_ui_with_size

ui only size: 1009.7151146829128 GB
including pre-aggregation size: 1541.5536647848785 GB
ratio: 1.5267213913787774


,variable,start_year,end_year,max_lat,min_lat,max_lon,min_lon,temporal_resolution,spatial_resolution,ui_id,size,agg_size,ratio
0,temperature,1981,2020,90,0,0,-180,day,0.50,1,4.381298,5.671680,1.294521
1,temperature,1981,2020,90,0,180,0,month,0.50,2,0.144043,0.195058,1.354167
2,temperature,1981,2020,0,-90,0,-180,day,0.25,3,17.525192,23.821057,1.359247
3,temperature,1981,2020,0,-90,180,0,day,0.25,4,17.525192,23.821057,1.359247
4,temperature,1941,1980,90,0,0,-180,month,0.50,5,0.144043,0.195058,1.354167
5,temperature,1941,1980,90,0,180,0,hour,0.25,6,140.201533,295.461527,2.107406
6,temperature,1941,1980,0,-90,0,-180,day,0.25,7,17.525192,23.821057,1.359247
7,temperature,1941,1980,0,-90,180,0,day,0.25,8,17.525192,23.821057,1.359247
8,precipitation,1981,2020,90,0,0,-180,hour,0.50,9,105.151150,137.110617,1.303938
9,precipitation,1981,2020,90,0,180,0,day,0.25,10,17.525192,23.821057,1.359247
